In [ ]:
# imports for running algorithm and printing
using LinearAlgebra
using DelimitedFiles
using SparseArrays

In [ ]:
# INPUT: directory to data file (***data file must be in adjacency list format***)
# OUTPUT: 2-tuple (array_of_page_names, probability_matrix)
function process_web(file_name::String)
    # open file
    f = open(file_name)

    # get number of vertices in graph (i.e. number of pages in Web)
    n = parse(Int64, readline(f))

    # initialize adjacency matrix and array of website names
    names = Array{String, 1}(undef, n)
    P = zeros(Float64, n, n)

    # fill adjacency matrix and name array
    for i in 1 : n
        # get line
        line = readline(f)

        # get the page index (1-indexed)
        page_i = parse(Int64, line[1:findfirst(isequal(' '), line)]) + 1

        # get the page name
        line = line[findfirst(isequal(' '), line) + 1 : length(line)]
        space_index = findfirst(isequal(' '), line)
        page_name = line[1 : (space_index == nothing ? length(line) : space_index)]
        names[page_i] = page_name

        # fill matrix wherever website #page_num goes 
        # if it does not go anywhere, website is a "sink", then make it go everywhere
        if space_index == nothing
            P[:, page_i] += ones(n, 1)
        else
            while space_index != nothing
                # update line to next number
                line = line[space_index + 1 : length(line)]
                space_index = findfirst(isequal(' '), line)

                # if no number in remainder of line, break
                if (!occursin(r"\d", line))
                    break
                end

                # mark row j in column i of P
                end_indx = (space_index == nothing ? length(line) : space_index)
                num_str = line[1 : end_indx]
                page_j = parse(Int64, num_str) + 1
                P[page_j , page_i] = 1
            end
        end 
    end

    # divide each column by sum of columns to get probabilities
    for i in 1 : n
        P[: , i] /= sum(P[:, i])
    end

    # close file
    close(f)

    # return
    return names, P
end

In [ ]:
# INPUT: given probability/rankdom_walk matrix and very small epsilon value
# OUTPUT: converts ranks of websites by iterating through random walk until convergence less than epsilon is reached
function compute_pageranks(P, eps::Float64)
    # initialize P as sparse, initialize starting rank vector
    num_pages = Int64(sqrt(length(P)))
    P = sparse(P)
    x_0 = ones(num_pages) / num_pages
    x = x_0
    delta = eps + 1
    beta = 0.85
    
    # iterate through random walk (do power method) until certain convergence is reached
    while delta >= eps 
        x_new = beta * P * x + (1 - beta) * x_0
        delta = norm(x_new - x)
        x = x_new
    end
    
    # return resulting ranks vector
    return x
end

In [ ]:
# INPUT: given an array of names and ranks, where website with name names[i] has rank ranks[i]
# OUTPUT: prints website names with respective ranks, greatest to least (no return value)
function print_ranks(names::Array{String, 1}, ranks::Array{Float64, 1})
    # argsort ranks
    indxs = sortperm(ranks)
    
    # pretty print
    println()
    println("========================")
    println("RANKS: Highest to Lowest")
    println("========================")
    
    # print websites along with ranks, greatest to least
    for i in length(names):-1:1
        pos = length(names) + 1 - i
        print("$pos. ")
        print(names[indxs[i]])
        print("\t")
        println(round.(ranks[indxs[i]], digits=10))
    end
end

In [ ]:
# give dir to data file -- contains adjacency list representation of web
# provide var for convergence
web_dir = "Webs/100_web.txt"
eps = 0.00001

# calculate probability matrix and get names of websites
# then compute the rank vectors
names, P = process_web(web_dir)
@time ranks = compute_pageranks(P, eps)

# print the ranks greatest to least
print_ranks(names, ranks)